# Social Security cap and `expanded_income`

Why does raising the Social Security cap change `expanded_income`?

In [1]:
import taxcalc as tc
import pandas as pd

In [2]:
recs = tc.Records.cps_constructor()

In [3]:
REMOVE_SS_CAP_REFORM = {'SS_Earnings_c': {2019: 1e+16}}

In [4]:
base = tc.Calculator(tc.Policy(), recs)
base.advance_to_year(2019)
base.calc_all()

In [5]:
pol = tc.Policy()
pol.implement_reform(REMOVE_SS_CAP_REFORM)
reform = tc.Calculator(pol, recs)
reform.advance_to_year(2019)
reform.calc_all()

As expected, `aftertax_income` falls, by $89 billion.

In [6]:
def chg(metric):
    return reform.weighted_total(metric) - base.weighted_total(metric)

In [7]:
METRICS = ['aftertax_income', 'expanded_income', 'combined', 'iitax',
           'payrolltax', 'ptax_was']

In [8]:
chgs_b = pd.Series([chg(i) for i in METRICS], index=METRICS) / 1e9
chgs_b.astype(int)

aftertax_income    -88
expanded_income     63
combined           152
iitax               -5
payrolltax         157
ptax_was           127
dtype: int64

The rise in expanded income is half the change in payroll taxes from wages and salaries.

In [9]:
chg('expanded_income') / chg('ptax_was')

0.4999999999999914

Using Comp with the PUF (https://www.compmodels.org/PSLmodels/Tax-Brain/41139/), the payroll tax liability rises by $191 billion, and there's virtually no change to income tax liability.

Comp is currently having issues, so can't get the changes it says from the CPS.